### roboflow의 tfrecord 파일을 갖고 training 시키는 코드

### 조건!
1. tensorflow 2 이상의 버전
2. module 에러나면 pip install 해주면 됨! 안될경우 pip install 모듈이름 --user
3. !tar 같은 명령어 안되면 걍 수동으로 해주면 됨

In [2]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
#import tensorflow_hub as hub
print("TF version:", tf.__version__)
#print("Hub version:", hub.__version__) 노 필요
print("GPU is",tf.test.is_gpu_available())
print("GPU name",tf.test.gpu_device_name())
#print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is True
GPU name /device:GPU:0


In [25]:
# 이미 된 부분
#%cd C:\Users\IVPL-D14\models\research
#!protoc C:\Users\IVPL-D14\models\research\object_detection\protos\*.proto --python_out=.

C:\Users\IVPL-D14\models\research


C:\Users\IVPL-D14\models\research\object_detection\protos\anchor_generator.proto: File does not reside within any path specified using --proto_path (or -I).  You must specify a --proto_path which encompasses this file.  Note that the proto_path must be an exact prefix of the .proto file names -- protoc is too dumb to figure out when two paths (e.g. absolute and relative) are equivalent (it's harder than you think).


In [41]:
# 죽어도 안됨.. ㅋ 그냥 setup.py 직접 copy하도록!
#%cd C:/Users/IVPL-D14/models/research
#%xcopy C:/Users/IVPL-D14/models/research/object_detection/packages/tf2/setup.py C:/Users/IVPL-D14/models/research
    
#!python -m pip install .

C:\Users\IVPL-D14\models\research


UsageError: Line magic function `%xcopy` not found.


In [42]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.builders import model_builder  
# 이건 test해보려는 건데 tensor ver1에서만 된다. 굳이 필요없으니 생략하자
%matplotlib inline

### implement basic functions

In [43]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)

## 만든 roboflow 경로를 삽입

In [8]:
## 경로는 임의로 정하기
%cd C:/Users/IVPL-D14
!curl -L "https://app.roboflow.com/ds/gpVc0f5POA?key=2vXO6YAmMW" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

C:\Users\IVPL-D14


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   891    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   891  100   891    0     0    891      0  0:00:01  0:00:01 --:--:--   838

100   252  100   252    0     0    252      0  0:00:01  0:00:01 --:--:--   252
100   252  100   252    0     0    252      0  0:00:01  0:00:01 --:--:--     0
curl: (6) Could not resolve host: ;

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: unzip

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: roboflow.zip;

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl:

In [14]:
%pwd
%ls -al train

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DCD6-F914

 C:\Users\IVPL-D14 디렉터리


 C:\Users\IVPL-D14\train 디렉터리

2020-12-21  오후 06:00    <DIR>          .
2020-12-21  오후 06:00    <DIR>          ..
2020-12-21  오후 06:00        12,188,397 knife.tfrecord
2020-12-21  오후 06:00                65 knife_label_map.pbtxt
               2개 파일          12,188,462 바이트
               2개 디렉터리  372,881,281,024 바이트 남음


파일을 찾을 수 없습니다.


In [10]:
test_record_fname = 'C:/Users/IVPL-D14/models/valid/knife.tfrecord'
train_record_fname = 'C:/Users/IVPL-D14/models/train/knife.tfrecord'
label_map_pbtxt_fname = 'C:/Users/IVPL-D14/models/train/knife_label_map.pbtxt'

In [56]:
## 용량 너무 크다는 에러때문에 batch_size를 대폭 축소함

MODELS_CONFIG = {
    'efficientdet-d0': {
    'model_name': 'efficientdet_d0_coco17_tpu-32',
    'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
    'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    'batch_size': 16
    },
    'efficientdet-d1': {
    'model_name': 'efficientdet_d1_coco17_tpu-32',
    'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
    'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
    'batch_size': 16
    },
    'efficientdet-d2': {
    'model_name': 'efficientdet_d2_coco17_tpu-32',
    'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
    'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
    'batch_size': 16
    },
    'efficientdet-d3': {
    'model_name': 'efficientdet_d3_coco17_tpu-32',
    'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
    'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
    'batch_size': 16
    }
}

chosen_model = 'efficientdet-d0'
num_steps = 20000 
num_eval_steps = 500  
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] 

### download pre-trained wieghts from web

In [23]:
%mkdir C:\Users\IVPL-D14\models\research\deploy
%cd C:/Users/IVPL-D14/models/research/deploy

C:\Users\IVPL-D14\models\research\deploy


In [92]:
import wget

In [41]:
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
wget.download(download_tar)
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

  6% [....                                                                    ]  1851392 / 30736482

 13% [.........                                                               ]  4128768 / 30736482

 20% [...............                                                         ]  6447104 / 30736482

 28% [....................                                                    ]  8626176 / 30736482

 34% [.........................                                               ] 10715136 / 30736482

...

 75% [......................................................                  ] 23199744 / 30736482

 82% [...........................................................             ] 25419776 / 30736482

 87% [...............................................................         ] 26935296 / 30736482

 94% [....................................................................    ] 29179904 / 30736482

100% [........................................................................] 30736482 / 30736482

In [42]:
%cd C:/Users/IVPL-D14/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
wget.download(download_config)

C:\Users\IVPL-D14\models\research\deploy
100% [................................................................................] 4630 / 4630

'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

### prepare some info for training

In [60]:
pipeline_fname = 'C:/Users/IVPL-D14/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = 'C:/Users/IVPL-D14/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

In [58]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=3, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [61]:
import re

%cd C:\Users\IVPL-D14\models\research\deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    s = re.sub('fine_tune_checkpoint: ".*?"','fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
# tfrecord files train and test.
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
# label_map_path
    s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname),s)
# Set training batch_size.
    s = re.sub('batch_size: [0-9]+','batch_size: {}'.format(batch_size), s)
# Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+','num_steps: {}'.format(num_steps), s)
# Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+','num_classes: {}'.format(num_classes), s)
#fine-tune checkpoint type
    s = re.sub('fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    f.write(s)

C:\Users\IVPL-D14\models\research\deploy
writing custom configuration file


In [53]:
%pycat C:\Users\IVPL-D14\models\research\deploy\pipeline_file.config

In [53]:
pipeline_file = 'C:/Users/IVPL-D14/models/research/deploy/pipeline_file.config'
model_dir = 'C:/Users/IVPL-D14/training/'

### 만약 tensorflow 이상한 에러가 뜬다면 아래 import 수행하기

In [93]:
#import tensorflow.compat.v1 as tf

In [50]:
%ls -al

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DCD6-F914

 C:\Users\IVPL-D14\models\research 디렉터리



파일을 찾을 수 없습니다.


## 학습!
#### 학습될때 딱히 colab 처럼 아래에 진행상황이 뜨지않고 [*] 인 상태이므로 혹시 모르면 
#### 로컬폴더가서  ckpt 파일이 생성이 되고있는지 확인하면 된다

In [62]:
!python C:/Users/IVPL-D14/models/research/object_detection/model_main_tf2.py \
        --pipeline_config_path={pipeline_file} \
        --model_dir={model_dir} \
        --alsologtostderr \
        --num_train_steps=20000 \
        --sample_1_of_n_eval_examples=1 \
        --num_eval_steps={num_eval_steps}

2020-12-22 23:59:17.193799: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-22 23:59:21.110667: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-22 23:59:21.111295: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2020-12-22 23:59:21.134825: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 238.66GiB/s
2020-12-22 23:59:21.134854: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-22 23:59:21.145435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
2020-12-

W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._head_scope_conv_layers.BoxPredictionTower
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._head_scope_conv_layers.ClassPredictionTower
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._box_prediction_head._box_encoder_layers.0
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._prediction_heads.class_predictions_with_background._class_predictor_layers
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.box_encodings.0
W1223 00:00:10.902777 11728 util.py:161] Unresolved object in checkpoint:

W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.1.4.moving_mean
W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.1.4.moving_variance
W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.1.7.axis
W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.1.7.gamma
W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tower_layers_for_heads.class_predictions_with_background.1.7.beta
W1223 00:00:10.915773 11728 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._base_tow

In [68]:
%load_ext tensorboard
%tensorboard --logdir C:/Users/IVPL-D14/training/train
    ## 이부분 에러남.. 안되서 일단 냅둠

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4112), started 0:03:55 ago. (Use '!kill 4112' to kill it.)

In [70]:
%ls  C:\Users\IVPL-D14\training

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DCD6-F914

 C:\Users\IVPL-D14\training 디렉터리

2020-12-23  오전 02:02    <DIR>          .
2020-12-23  오전 02:02    <DIR>          ..
2020-12-23  오전 02:02               707 checkpoint
2020-12-23  오전 01:26        44,560,824 ckpt-17.data-00000-of-00001
2020-12-23  오전 01:26            89,701 ckpt-17.index
2020-12-23  오전 01:32        44,560,824 ckpt-18.data-00000-of-00001
2020-12-23  오전 01:32            89,701 ckpt-18.index
2020-12-23  오전 01:38        44,560,824 ckpt-19.data-00000-of-00001
2020-12-23  오전 01:38            89,701 ckpt-19.index
2020-12-23  오전 01:44        44,560,824 ckpt-20.data-00000-of-00001
2020-12-23  오전 01:44            89,701 ckpt-20.index
2020-12-23  오전 01:50        44,560,824 ckpt-21.data-00000-of-00001
2020-12-23  오전 01:50            89,701 ckpt-21.index
2020-12-23  오전 01:56        44,560,824 ckpt-22.data-00000-of-00001
2020-12-23  오전 01:56            89,701 ckpt-22.index
2020-12-23  오전 02:02        44,560,824 ckpt-23.data-00000-of-00001


In [71]:
import re
import numpy as np
output_directory = 'C:/Users/IVPL-D14/fine_tuned_model'
#place the model weights you would like to export here
last_model_path = 'C:/Users/IVPL-D14/training'
print(last_model_path)
!python C:/Users/IVPL-D14/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

C:/Users/IVPL-D14/training


2020-12-23 11:18:31.590581: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-23 11:18:35.482108: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-23 11:18:35.482711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2020-12-23 11:18:35.501068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 238.66GiB/s
2020-12-23 11:18:35.501098: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-23 11:18:35.516445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
2020-12-

In [73]:
#check check
%ls -al C:\Users\IVPL-D14\fine_tuned_model\saved_model

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DCD6-F914

 C:\Users\IVPL-D14\models\research\deploy 디렉터리


 C:\Users\IVPL-D14\fine_tuned_model\saved_model 디렉터리

2020-12-23  오전 11:21    <DIR>          .
2020-12-23  오전 11:21    <DIR>          ..
2020-12-23  오전 11:21    <DIR>          assets
2020-12-23  오전 11:21        26,078,241 saved_model.pb
2020-12-23  오전 11:21    <DIR>          variables
               1개 파일          26,078,241 바이트
               4개 디렉터리  351,960,776,704 바이트 남음


파일을 찾을 수 없습니다.


In [74]:
%ls -al C:\Users\IVPL-D14\fine_tuned_model\saved_model\variables

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: DCD6-F914

 C:\Users\IVPL-D14\models\research\deploy 디렉터리


 C:\Users\IVPL-D14\fine_tuned_model\saved_model\variables 디렉터리

2020-12-23  오전 11:21    <DIR>          .
2020-12-23  오전 11:21    <DIR>          ..
2020-12-23  오전 11:21        22,614,874 variables.data-00000-of-00001
2020-12-23  오전 11:21            39,718 variables.index
               2개 파일          22,654,592 바이트
               2개 디렉터리  351,960,473,600 바이트 남음


파일을 찾을 수 없습니다.


In [75]:
import os
output_directory = 'C:/Users/IVPL-D14/fine_tuned_model/saved_model/'
pb_fname = os.path.join(os.path.abspath(output_directory), "saved_model.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

### download 하는 부분!
google.colab 모듈을 불러올수 없는 문제도 있지만.. 이미 로컬에 저장되어있으므로 굳이
다운받지 않고 사용하면 됨..!

In [86]:
#label_map_pbtxt_fname = 'C:/Users/IVPL-D14/models/train/knife_label_map.pbtxt'
#from google.colab import files
#files.download(label_map_pbtxt_fname)

### 이 명령어도 듣지 않는다.
대신 앞서 만든 deploy 폴더안의 tar 파일을 fine_tuned_model에 가져와서 압축해제 해주면 된다!


#%pwd

#!tar -cvzf trained_model_large_original_20000.tar.gz C:/Users/IVPL-D14/fine_tuned_model/

In [90]:
## test 해주는 부분 돌리면 됨.. !